In [1]:
import json
import networkx as nx
import numpy as np
import math
from numpy.linalg import norm

In [2]:
def showGraph(G):
    pos = {}
    for node in G.nodes(data = True):
        pos[node[0]] = [int(p) for p in node[1]['position'][0:2]]
    nx.draw(G, pos = pos)

In [3]:
def createGraph(graph):
    pass
    # G = nx.Graph()
    # for node in graph['node']:
    #     G.add_node(node['storeName'], 
    #         storeName = node['storeName'], 
    #         position = (node['posX'], node['posY']))
    # for edge in graph['edge']:
        # x1,y1,z1 = map(int,G.nodes[edge['node'][0]]['position'])
        # x2,y2,z2 = map(int,G.nodes[edge['node'][1]]['position'])
        # G.add_edge(edge['node'][0], edge['node'][1], weight = int(abs((x1-x2) +(y1-y2) +(z1-z2))))
    # return G

In [4]:
def readJson(path,nodeType):
    G = []
    nodes = []
    f = open(path)
    graph = json.load(f, parse_int=str)
    count = 1
    for node in graph['storeNodes']:
        nodeId = nodeType + str(count)
        nodePos = np.array([int(node['posX']),int(node['posY'])])
        G.append([nodeId,nodePos])
        nodes.append([nodeId,node['storeName']])
        count += 1
    return (G,nodes)

In [5]:
sGraph, Nodes = readJson("./Jsons/S3.json","store")
print('sGraph')
print(sGraph)
cGraph, Nodes = readJson("./Jsons/C3.json","C")
print('cGraph')
print(cGraph)

sGraph
[['store1', array([5, 2])], ['store2', array([10,  2])], ['store3', array([16,  4])], ['store4', array([16, 11])], ['store5', array([ 9, 13])], ['store6', array([7, 8])], ['store7', array([12,  8])], ['store8', array([3, 8])]]
cGraph
[['C1', array([5, 8])], ['C2', array([10, 11])], ['C3', array([14,  8])], ['C4', array([10,  5])]]


In [6]:
def alignNodes(G):
    i = 0
    for nodeId_i, pos_i in G:
        i += 1
        for nodeId_j, pos_j in G[i:]:
            for axis in range(2):
                if(1 <= abs(pos_i[axis] - pos_j[axis]) < 2):
                    pos_i[axis] = pos_j[axis] = (pos_i[axis] + pos_j[axis])/2
    
    i = 0
    for nodeId_i, pos_i in G:
        i += 1
        for nodeId_j, pos_j in G[i:]:
            for axis in range(2):
                if(1 <= abs(pos_i[axis] - pos_j[axis]) < 2):
                    pos_i[axis] = pos_j[axis] = int((pos_i[axis] + pos_j[axis])//2)
    return G

In [7]:
alignedStoreNodes = alignNodes(sGraph)
print("alignedStoreNodes")
print(alignedStoreNodes)
alignedCorridorNodes = alignNodes(cGraph)
print("alignedCorridorNodes¸")
print(alignedCorridorNodes)

alignedStoreNodes
[['store1', array([5, 2])], ['store2', array([9, 2])], ['store3', array([16,  4])], ['store4', array([16, 11])], ['store5', array([ 9, 13])], ['store6', array([7, 8])], ['store7', array([12,  8])], ['store8', array([3, 8])]]
alignedCorridorNodes¸
[['C1', array([5, 8])], ['C2', array([10, 11])], ['C3', array([14,  8])], ['C4', array([10,  5])]]


In [8]:
def getCorridors(G):
    G = sorted(G,key=lambda x: x[1][0])
    Corners = []
    i = 0
    for nodeId_i, pos_i in G:
        i += 1
        for nodeId_j, pos_j in G[i:]:
            if(not(pos_i[0] == pos_j[0] or pos_i[1] == pos_j[1])):
                if(not([pos_i[0],pos_j[1]] in Corners)):
                    Corners.append([pos_i[0],pos_j[1]])
                else: 
                    Corners.pop(Corners.index([pos_i[0],pos_j[1]]))

                if(not([pos_j[0],pos_i[1]] in Corners)):
                    Corners.append([pos_j[0],pos_i[1]])
                else:
                    Corners.pop(Corners.index([pos_j[0],pos_i[1]]))
    corridorCorners = []
    count = 1
    for node in Corners:
        cId = "C" + str(count)
        corridorCorners.append([cId,np.array(node)])
        count += 1

    Corridors = []
    for i in range(len(Corners)):
            for j in range(i+1,len(Corners)):
                if(Corners[i][0]==Corners[j][0]):
                    Corridors.append([0,np.array(Corners[i]),np.array(Corners[j])])
                if(Corners[i][1]==Corners[j][1]):
                    Corridors.append([1,np.array(Corners[i]),np.array(Corners[j])])
    return corridorCorners, Corridors

In [9]:
corridorCorners, corridors= getCorridors(alignedCorridorNodes)
print('corridorCorners')
print(corridorCorners)
print('corridors')
print(corridors)

corridorCorners
[['C1', array([ 5, 11])], ['C2', array([5, 5])], ['C3', array([14, 11])], ['C4', array([14,  5])]]
corridors
[[0, array([ 5, 11]), array([5, 5])], [1, array([ 5, 11]), array([14, 11])], [1, array([5, 5]), array([14,  5])], [0, array([14, 11]), array([14,  5])]]


In [10]:
# d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)
def getCorridorNodes(storeNodes,Corridors,CorridorNodes):
    for sId,storeNode in storeNodes:
        distance = [9999]
        newNode = np.copy(storeNode)            
        for [axis,p1,p2] in Corridors:
            d = norm(np.cross(p2-p1, p1-storeNode))/norm(p2-p1)
            if(distance[0] > d):
                distance = [d,axis,p1]
            if(d == 0):
                if(norm(p1-storeNode)>norm(p2-storeNode)):
                    newNode = np.copy(p2)
                else:
                    newNode = np.copy(p1)
                break
        newNode[distance[1]] = distance[2][distance[1]]
        flag = 1
        for cId ,cnode in CorridorNodes:
            if(np.array_equal(newNode,cnode)):
                flag = 0
                break
        if(flag):
            cId = "C" + str(len(CorridorNodes) + 1)
            CorridorNodes.append([cId,newNode])
    return CorridorNodes

In [11]:
corridorNodes = getCorridorNodes(alignedStoreNodes,corridors,corridorCorners)
print(corridorNodes)
print(len(corridorNodes))

[['C1', array([ 5, 11])], ['C2', array([5, 5])], ['C3', array([14, 11])], ['C4', array([14,  5])], ['C5', array([9, 5])], ['C6', array([16,  5])], ['C7', array([ 9, 11])], ['C8', array([5, 8])], ['C9', array([14,  8])]]
9


In [12]:
def connectSNodes(corridorNodes,storeNodes):
    edges = []
    for sId, p1 in storeNodes:
        distance = [9999]
        for cId, p2 in corridorNodes:
            if((p1 == p2).any()):
                d = int(norm(p2-p1))
                if(distance[-1]>d):
                    distance = [sId,cId,p1,p2,d]
        edges.append(distance)
    return edges

In [13]:
sEdges = connectSNodes(corridorNodes,alignedStoreNodes)
print(sEdges)
print(len(sEdges),len(alignedStoreNodes))

[['store1', 'C2', array([5, 2]), array([5, 5]), 3], ['store2', 'C5', array([9, 2]), array([9, 5]), 3], ['store3', 'C6', array([16,  4]), array([16,  5]), 1], ['store4', 'C3', array([16, 11]), array([14, 11]), 2], ['store5', 'C7', array([ 9, 13]), array([ 9, 11]), 2], ['store6', 'C8', array([7, 8]), array([5, 8]), 2], ['store7', 'C9', array([12,  8]), array([14,  8]), 2], ['store8', 'C8', array([3, 8]), array([5, 8]), 2]]
8 8


In [14]:
def connectCNodes(corridorNodes,storeNodes):
    edges = []
    i = 0
    for cId_i, p1 in corridorNodes:
        i += 1
        distance = []
        for cId_j, p2 in corridorNodes[i:]:
            if((p1 == p2).any()):
                d = int(norm(p2-p1))
                distance.append([cId_i,cId_j,p1,p2,d])
        distance = sorted(distance,key=lambda x: x[-1])
        try:
            edges.append(distance[0])
            edges.append(distance[1])
        except:
            pass
    return edges

In [15]:
cEdges = connectCNodes(corridorNodes,alignedStoreNodes)
print(cEdges)
print(len(cEdges),len(corridorNodes))

[['C1', 'C8', array([ 5, 11]), array([5, 8]), 3], ['C1', 'C7', array([ 5, 11]), array([ 9, 11]), 4], ['C2', 'C8', array([5, 5]), array([5, 8]), 3], ['C2', 'C5', array([5, 5]), array([9, 5]), 4], ['C3', 'C9', array([14, 11]), array([14,  8]), 3], ['C3', 'C7', array([14, 11]), array([ 9, 11]), 5], ['C4', 'C6', array([14,  5]), array([16,  5]), 2], ['C4', 'C9', array([14,  5]), array([14,  8]), 3], ['C5', 'C7', array([9, 5]), array([ 9, 11]), 6], ['C5', 'C6', array([9, 5]), array([16,  5]), 7], ['C8', 'C9', array([5, 8]), array([14,  8]), 9]]
11 9
